In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
import pyspark.sql.functions as func
from pyspark.sql.functions import countDistinct

In [2]:
#reading in data again
df_new = spark.read.parquet("gs://ds-url-catag/plenty_stickers_data/aggregate_data/")
df_demo = spark.read.parquet("gs://ds-user-demographic/parquet/tastes_mau_1024.parquet")

#joining to get the demographic data
from pyspark.sql.functions import col
active_base  = df_new.alias('a').join(df_demo.alias('b'),col('b.regid') == col('a.user_id_n')).select([col('time'),col('City'),col('Connection'),col('Country'),col('Device_Brand'),col('Device_Model'),col('New_User'),col('OS_Version'),col('app_name'),col('carrier'),col('license_type'),col('a.platform'),col('sticker_id'),col('sticker_pack_id'),col('user_id_n')]+[col('language'),col('age_group'),col('gender')])

#joining in to get the sticker category data
stick_category = spark.read.parquet("gs://ds-url-catag/Stickers/proces_sticker_bycategory/final/*/*.parquet")

In [6]:
#sticker categories to user 
with_category = active_base.join(stick_category,stick_category.id==active_base.sticker_pack_id).select([active_base.user_id_n,active_base.sticker_pack_id]+[final_stickcat.id,final_stickcat.Name,final_stickcat.category])

In [ ]:
#calculating scores 
category_Scores = with_category.groupby('user_id_n','category').agg(func.count("*").alias('category_scores'))
Total_Scores = with_category.groupby('user_id_n').agg(func.count("*").alias('Total_scores'))

#joing to get the full data file 
Final_affinty_file = category_Scores.join(Total_Scores,Total_Scores.user_id_n==category_Scores.user_id_n,'left').select[()]


In [ ]:
# #joining to get the sticker type data
# Final_stickers_types = sqlContext.read.parquet("gs://ds-url-catag/Stickers/stick_bytype/agg_proc_stickertypes/")
# #sticker types used by users
# data_join = active_base.join(Final_stickers_types,Final_stickers_types.id==active_base.sticker_pack_id).select([active_base.sticker_pack_id,active_base.user_id_n]+[Final_stickers_types.id,Final_stickers_types.type])
# data_join = data_join.dropDuplicates()